In [1]:
import pandas as pd

# Cargar el CSV generado en el preprocesamiento
df = pd.read_csv("../data/clientes_preprocesado.csv")
df.head()

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,Generacion,Ingresos_anuales_brutos,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno,comunidad_autonoma
0,CLI10,0,1,1.036719,1,4,0,1,3,1,1.052557,-0.100034,1.226055,1,-0.692744,0.0,Castilla y Leon
1,CLI100,1,1,0.873632,1,5,0,2,2,1,1.231266,-0.255823,0.170131,1,0.416626,5.0,Comunidad Valenciana
2,CLI1000,1,1,0.927994,1,0,0,2,3,1,-0.547295,0.055288,-0.533818,0,0.732628,3.0,Castilla La Mancha
3,CLI10000,1,0,-0.811600,0,3,1,1,4,3,-0.889922,0.003358,-0.533818,0,1.653741,4.0,Castilla y Leon
4,CLI10001,1,1,0.221284,0,7,0,2,2,1,-0.742655,-0.118280,-0.533818,1,-0.860830,0.0,Catalunya
